# Chapter 6. 다양한 데이터 타입 다루기
- 스파크의 표현식을 만드는 방법
- 스파크의 다양한 데이터 타입을 다루는 방법
  - 불리언 타입
  - 수치 타입
  - 문자열 타입
  - date와 timestamp 타입
  - null값 다루기
  - 복합 데이터 타입
  - 사용자 정의 함수

## 6.1 API는 어디서 찾을까
- 스파크는 현재 활발하게 성장하고 있는 프로젝트
  - 데이터 변환용 함수를 어떻게 찾는지 알아야 함
  - 데이터 변환용 함수를 찾기 위해 핵심적으로 보아야 할 부분
    - DataFrame(Dataset) 메서드
      - DataFrame은 Row 타입을 가진 Dataset임, 결국에는 Dataset 메서드를 만나게 됨
      - DataFrameStatFunctions와 DataFrameNaFunctions 등 Dataset의 하위 모듈은 다양한 메서드를 제공함
        - 해당 메서드를 사용해 여러 가지 문제를 해결할 수 있음
        - DataFrameStatFuctions : 다양한 통계적 함수를 제공함
        - DataFrameNaFuctions : null 데이터를 다루는 데 필요한 함수 제공
    - Column 메서드
      - alias나 contains 같이 컬럼과 관련된 여러 가지 메서드를 제공함
      - org.apache.spark.sql.functions
        - 데이터 타입과 관련된 다양한 함수를 제공함
        - 해당 패키지를 많이 쓰이므로 보통 전체 패키지를 임포트하고 사용함
- 위의 함수들은 대부분의 SQL과 분석 시스템에서도 찾아볼 수 있는 함수
- 모든 함수는 데이터 로우의 특정 포맷이나 구조를 다른 형태로 변환하기 위해 존재함
  - 함수를 사용해 더 많은 로우를 만들거나 줄일 수 있음

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("sample").master("local[*]").getOrCreate()

In [2]:
#분석에 사용할 DataFrame을 생성하는 예제
df=spark.read.format("csv")\
    .option("header","true")\
    .option("inferSchema","true")\
    .load("./Spark-The-Definitive-Guide-master/data/retail-data/by-day/2010-12-01.csv")
df.printSchema()
df.createOrReplaceTempView("dfTable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



## 6.2 스파크 데이터 타입으로 변환하기
- 프로그래밍 언어의 고유 데이터 타입을 스파크 데이터 타입으로 변환
- 스파크 데이터 타입으로 변환하는 방법
  - lit 함수를 사용함
    - 다른 언어의 데이터 타입을 스파크 데이터 타입에 맞게 변환함
  - SQL에서는 스파크 데이터 타입으로 변환할 필요가 없으므로 값을 직접 입력해 사용함
    - SELECT 5,"five",5.0

In [3]:
from pyspark.sql.functions import lit
df.select(lit(5),lit('five'),lit(5.0))

DataFrame[5: int, five: string, 5.0: double]

## 6.3 불리언 데이터 타입 다루기
- 불리언
  - 모든 필터링 작업의 기반이므로 데이터 분석에 필수적
  - 불리언 구문 : and, or, ture, false
  - 불리언 구문을 사용해 true 또는 false로 평가되는 논리 문법을 만듦
    - 논리 문법 : 데이터 로우를 필터링할 때 필요조건의 일치와 불일치를 판별하는 데 사용됨
  - 불리언 식에는 일치 조건뿐만 아니라 작다, 크다와 같은 비교 연산조건을 사용할 수 있음

In [4]:
from pyspark.sql.functions import col
df.where(col("InvoiceNO")!=536365)\
    .select("InvoiceNo","Description")\
    .show(5,False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



- and 메서드나 or 메서드를 사용해 불리언 표현식을 여러 부분에 지정할 수 있음
  - 불리언 표현식을 사용하는 경우 항상 모든 표현식을 and 메서드로 묶어 차례대로 필터를 적용해야 함
    - 차례대로 필터를 적용해야 하는 이유
         - 불리언 문을 차례대로 표현하더라도 스파크는 내부적으로 and 구문을 필터 사이에 추가해 모든 필터를 하나의 문장으로 변환함
         - 그런 다음 동시에 모든 필터를 처리함
         - 원한다면 and 구문으로 조건문을 만들 수도 있음
         - 하지만 차례로 조건을 나열하면 이해하기 쉽고 읽기도 편해짐
- or 구문을 사용할 때는 반드시 동일한 구문에 조건을 정의해야 함

In [5]:
from pyspark.sql.functions import instr
priceFilter=col("UnitPrice")>600
descripFilter=instr(df.Description,"POSTAGE")>=1
df.where(df.StockCode.isin("DOT")).where(priceFilter|descripFilter).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



#### instr 함수란?
- sql에서는 문자열 속 문자 위치 찾기에서 사용됨
- INSTR 함수
  - 문자열에서 지정된 문자열을 검색해서 그 위치를 리턴하는 함수
  - 위치는 지정된 문자열이 타나내는 제일 첫 번째 위치를 리턴함
    - ex) INSTR('CONGRATULATIONS','AT',1,1) -> 'CONGRATULATIONS' 문자열에서 'AT' 문자열을 첫번째 문자부터 찾아서 'AT' 문자열이 처음 나오는 위치를 리턴해라. 첫번째 'AT'에서 'A'의 위치는 여섯번째이므로 6이 리턴

- 불리언 표현식을 필터링 조건에만 사용하는 것은 아님
  - 불리언 컬럼을 사용해 DataFrame을 필터링할 수도 있음

In [6]:
from pyspark.sql.functions import instr
DOTCodeFilter=col("StockCode")=="DOT"
priceFilter=col("UnitPrice")>600
descripFilter=instr(col("Description"),"POSTAGE")>=1
df.withColumn("isExpensive",DOTCodeFilter & (priceFilter|descripFilter))\
    .where("isExpensive")\
    .select("unitPrice","isExpensive").show(5)

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|   569.77|       true|
|   607.49|       true|
+---------+-----------+



- 필터를 반드시 표현식으로 정의할 필요는 없음
  - 별도의 작업 없이 컬럼명을 사용해 필터를 정의할 수도 있음

In [7]:
from pyspark.sql.functions import expr
df.withColumn("isExpensive",expr("NOT UnitPrice<=250"))\
    .where("isExpensive")\
    .select("Description","UnitPrice").show(5)

+--------------+---------+
|   Description|UnitPrice|
+--------------+---------+
|DOTCOM POSTAGE|   569.77|
|DOTCOM POSTAGE|   607.49|
+--------------+---------+



## 6.4 수치형 데이터 타입 다루기
- 카운트 : 빅데이터 처리에서 필터링 다음으로 많이 수행하는 작업
  - 수치형 데이터 타입을 사용해 연산 방식을 정의하기만 하면 됨
- pow
  - 수치형 함수
  - 표시된 지수만큼 컬럼의 값을 거듭제곱함

In [8]:
from pyspark.sql.functions import expr, pow
fabricatedQuantity=pow(col("Quantity")*col("UnitPrice"),2)+5
df.select(expr("CustomerId"),fabricatedQuantity.alias("realQuantity")).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [9]:
df.selectExpr(
"CustomerId",
"(POWER((Quantity*UnitPrice),2.0)+5) as realQuantity").show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



- 반올림도 자주 사용한느 수치형 작업 중 하나
  - 때로는 소수점 자리를 없애기 위해 Integer 데이터 타입으로 형변환하기도 함
  - 스파크는 정확한 계산이 가능한 함수를 제공함
  - 정밀도를 사용해 더 세밀한 작업을 수행할 수 있음
- 기본적으로 round 함수는 소수점 값이 정확히 중간값 이상이면 반올림함
- 내림은 bround 함수를 사용함

In [10]:
from pyspark.sql.functions import lit,round,bround
df.select(round(lit("2.5")),bround(lit("2.5"))).show(2)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 2 rows



- 두 컬럼 사이의 상관관계를 계산하는 것도 수치형 연산 작업 중 하나
  - 스파크에서도 DataFrame의 통계용 함수나 메서드를 사용해 피어슨 상관계수를 계산할 수 있음

In [11]:
from pyspark.sql.functions import corr
df.stat.corr("Quantity","UnitPrice")
df.select(corr("Quantity","UnitPrice")).show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+



- 하나 이상의 컬럼에 대한 요약 통계를 계산하는 작업 역시 자주 수행됨
  - describe : 요약 통계
    - 관련 컬럼에 대한 집계, 평균, 표준편차, 최솟값, 최댓값을 계산함
    - 통계 스키마는 변경될 수 있으므로 해당 메서드는 콘솔 확인용으로만 사용해야 함

In [12]:
df.describe().show()

+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|        InvoiceDate|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|               3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128|               null| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|               null|15.638659854603892|1854.4496996893627|          null|
|    min|           536365|             

- 정확한 수치가 필요하다면 함수를 임포트하고 해당 컬럼에 적용하는 방식으로 직접 집계를 수행할 수 있음

In [13]:
from pyspark.sql.functions import count,mean,stddev_pop,min,max

- StatFunctions 패키지
  - 다양한 통계 함수를 제공함
  - stat 속성을 사용해 접근할 수 있으며 다양한 통계값을 계산할 때 사용하는 DataFrame 메서드
  - ex) approxQuantile 메서드를 사용해 데이터의 백분위수를 정확하게 계산하거나 근사치를 계산할 수 있음

In [14]:
olName="UnitPrice"
quantileProbs=[0.5]
relError=0.05

df.stat.approxQuantile("UnitPrice",quantileProbs,relError)

[2.51]

- StatFunctions 패키지
  - 교차표나 자주 사용하는 항목 쌍을 확인하는 용도의 메서드도 제공함
  - 단, 연산 결과가 너무 크면 화면에 모두 보이지 않을 수 있음


In [15]:
df.stat.crosstab("StockCode","Quantity").show()

+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|StockCode_Quantity| -1|-10|-12| -2|-24| -3| -4| -5| -6| -7|  1| 10|100| 11| 12|120|128| 13| 14|144| 15| 16| 17| 18| 19|192|  2| 20|200| 21|216| 22| 23| 24| 25|252| 27| 28|288|  3| 30| 32| 33| 34| 36|384|  4| 40|432| 47| 48|480|  5| 50| 56|  6| 60|600| 64|  7| 70| 72|  8| 80|  9| 96|
+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|             22578|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 

In [16]:
#해당 열에서 빈번하게 나오는 요소들을 보여준다.
df.stat.freqItems(["StockCode","Quantity"]).show()

+--------------------+--------------------+
| StockCode_freqItems|  Quantity_freqItems|
+--------------------+--------------------+
|[90214E, 20728, 2...|[200, 128, 23, 32...|
+--------------------+--------------------+



- StatFunctions 패키지의 함수인 monotonically_increasing_id
  - 모든 로우에 고유 ID 값을 추가함
  - 모든 로우에 0부터 시작하는 고윳값을 생성함

In [17]:
from pyspark.sql.functions import monotonically_increasing_id
df.select(monotonically_increasing_id()).show(2)

+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
+-----------------------------+
only showing top 2 rows



- 스파크의 새로운 버전이 릴리스될 때마다 새로운 함수가 생겨남
- 새로 추가된 함수는 스파크 공식 문서에서 확인할 수 있음
  - ex) 불규칙적으로 데이터를 생성할 수 있는 rand() 함수나 radn() 함수 등 임의 데이터 생성 함수
    - 해당 함수들은 잠재적으로 결정론과 관련된 문제를 가지고 있음
    - 스파크 메일링 리스트에서 이와 관련된 논의를 찾아볼 수 있음
  - 최선 버전의 StatFunction 패키지는 블롬 필터링이나 스케칭 알고리즘 같은 여러 고급 기법과 관련된 함수를 제공함

## 6.5 문자열 데이터 타입 다루기
- 문자열을 다루는 작업은 거의 모든 데이터 처리 과정에서 발생함
  - 문자열을 다루는 방법 : 로그 파일에 정규 표현식을 사용해 데이터 추출, 데이터 치환, 문자열 존재 여부, 대/소문자 변환 처리 등의 작업을 할 수 있음

- 대/소문자 변환 작업
  - initcap 함수 : 주어진 문자열에서 공백으로 나뉘는 모든 단어의 첫 글자를 대문자로 변경함

In [18]:
from pyspark.sql.functions import initcap
df.select(initcap(col("Description"))).show()

+--------------------+
|initcap(Description)|
+--------------------+
|White Hanging Hea...|
| White Metal Lantern|
|Cream Cupid Heart...|
|Knitted Union Fla...|
|Red Woolly Hottie...|
|Set 7 Babushka Ne...|
|Glass Star Froste...|
|Hand Warmer Union...|
|Hand Warmer Red P...|
|Assorted Colour B...|
|Poppy's Playhouse...|
|Poppy's Playhouse...|
|Feltcraft Princes...|
|Ivory Knitted Mug...|
|Box Of 6 Assorted...|
|Box Of Vintage Ji...|
|Box Of Vintage Al...|
|Home Building Blo...|
|Love Building Blo...|
|Recipe Box With M...|
+--------------------+
only showing top 20 rows



- lower 함수 : 문자열 전체를 소문자로 변경함
- upper 함수 : 문자열 전체를 대문자로 변경함


In [19]:
from pyspark.sql.functions import lower,upper
df.select(col("Description"),
         lower(col("Description")),
         upper(lower(col("Description")))).show(2)

+--------------------+--------------------+-------------------------+
|         Description|  lower(Description)|upper(lower(Description))|
+--------------------+--------------------+-------------------------+
|WHITE HANGING HEA...|white hanging hea...|     WHITE HANGING HEA...|
| WHITE METAL LANTERN| white metal lantern|      WHITE METAL LANTERN|
+--------------------+--------------------+-------------------------+
only showing top 2 rows



- 문자열 주변의 공백을 제거하거나 추가하는 작업도 가능함
  - lpad,ltrim,rpad,rtrim,trim 함수

In [20]:
from pyspark.sql.functions import lit,ltrim,rtrim,rpad,lpad,trim
df.select(
    ltrim(lit("  HELLO  ")).alias("ltrim"),
    rtrim(lit("  HELOO  ")).alias("rtrim"),
    trim(lit("  HELLO  ")).alias("trim"),
    lpad(lit("HELLO"),3," ").alias("lp"),
    rpad(lit("HELLO"),10," ").alias("rp")).show(2)

+-------+-------+-----+---+----------+
|  ltrim|  rtrim| trim| lp|        rp|
+-------+-------+-----+---+----------+
|HELLO  |  HELOO|HELLO|HEL|HELLO     |
|HELLO  |  HELOO|HELLO|HEL|HELLO     |
+-------+-------+-----+---+----------+
only showing top 2 rows



- lpad 함수나 rpad 함수에 문자열의 길이보다 작은 숫자를 넘기면 문자열의 오른쪽부터 제거됨

### 6.5.1 정규 표현식
- 정규 표현식 : 문자열의 존재 여부를 확인하거나 일치하는 모든 문자열을 치환할 때 사용
  - 문자열에서 값을 추출하거나 다른 값으로 치환하는 데 필요한 규칙 모음을 정의할 수 있음
  - 스파크는 자바 정규 표현식이 가진 강력한 능력을 활용함
    - 자바 정규 표현식 문법은 보통 사용하는 언어의 문법과 약간 다르므로 운영 환경에서 정규 표현식을 사용하기 전에 다시 한 번 검토해야함
- 정규 표현식을 위해 regexp_extract 함수, regexp_replace 함수를 제공함
  - 이 함수들은 값을 추출하고 치환하는 역할을 함

In [21]:
#regexp_replace 함수를 사용해 'description' 컬럼의 값을 'COLOR'로 치환
from pyspark.sql.functions import regexp_replace
regex_string="BLACK|WHITE|RED|GREEN|BLUE"
df.select(
    regexp_replace(col("Description"),regex_string,"COLOR").alias("color_clean"),
    col("Description")).show(2)

+--------------------+--------------------+
|         color_clean|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
+--------------------+--------------------+
only showing top 2 rows



- 주어진 문자를 다른 문자로 치환해야 할 때도 있음
  - translate 함수로 문자 치환 가능
    - 해당 연산은 문자 단위로 이루어짐
    - 교체 문자열에서 색인된 문자에 해당하는 모든 문자를 치환함

In [22]:
from pyspark.sql.functions import translate
df.select(translate(col("Description"),"LEET","1337"),col("Description"))\
    .show(2)
#해당 예제의 경우 L=1, E=3, T=7로 치환됨

+----------------------------------+--------------------+
|translate(Description, LEET, 1337)|         Description|
+----------------------------------+--------------------+
|              WHI73 HANGING H3A...|WHITE HANGING HEA...|
|               WHI73 M37A1 1AN73RN| WHITE METAL LANTERN|
+----------------------------------+--------------------+
only showing top 2 rows



In [23]:
#처음 나타난 색상 이름을 추출하는 것과 같은 작업을 수행할 수도 있음
from pyspark.sql.functions import regexp_extract
extract_str="(BLACK|WHITE|RED|GREEN|BLUE)"
df.select(
    regexp_extract(col("Description"),extract_str,1).alias("color_clean"),
    col("Description")).show(2)

+-----------+--------------------+
|color_clean|         Description|
+-----------+--------------------+
|      WHITE|WHITE HANGING HEA...|
|      WHITE| WHITE METAL LANTERN|
+-----------+--------------------+
only showing top 2 rows



- 때로는 값 추출 없이 단순히 값의 존재 여부를 확인하고 싶을 때가 있음
  - contains 메서드를 사용함
    - 인수로 입력된 값이 컬럼의 문자열에 존재하는지 불리언 타입으로 반환함
  - instr 함수
    - 파이썬과 SQL에서는 해당 함수를 통해 값의 존재 여부를 확인함

In [24]:
from pyspark.sql.functions import instr
containsBlack=instr(col("Description"),"BLACK")>=1
containsWhite=instr(col("Description"),"WHITE")>=1
df.withColumn("hasSimpleColor",containsBlack|containsWhite)\
    .where("hasSimpleColor")\
    .select("Description").show(3,False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



- 동적으로 인수의 개수가 변하는 상황을 스파크가 처리하는 방법
  - varargs: 값 목록을 인수로 변환해 함수에 전달할 때
    - 이 기능을 사용해 임의 길이의 배열을 효율적으로 다룰 수 있음
    - ex) select 메서드와 varargs를 함께 사용해 원하는 만큼 동적으로 컬럼을 생성할 수 있음
  - 파이썬은 인수의 개수가 동적으로 변하는 상황을 아주 쉽게 해결할 수 있음
    - locate 함수 : 문자열의 위치(위치는 1부터 시작)를 정수로 반환
      - 그런 다음 위치 정보를 불리언 타입으로 변환함
      

In [25]:
from pyspark.sql.functions import expr,locate
simpleColors=["black","white","red","green","blue"]
def color_locator(column,color_string):
    return locate(color_string.upper(),column)\
            .cast("boolean")\
            .alias("is_"+color_string)
selectedColumns=[color_locator(df.Description,c) for c in simpleColors]
selectedColumns.append(expr("*")) #column 타입이어야 함

df.select(*selectedColumns).where(expr("is_white OR is_red"))\
    .select("Description").show(3,False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



- locate 함수는 쉽게 확장할 수 있음
  - 컬럼이나 불리언 필터를 프로그래밍 방식으로 생성할 수 있음
  - ex) locate 함수를 확장해 입력값의 최소공배수를 구하거나 소수 여부를 판별할 수 있음

## 6.6 날짜와 타임스탬프 데이터 타입 다루기
- 날짜와 시간은 프로그래밍 언어와 데이터베이스 분야의 변함없는 과제
  - 계속해서 시간대를 확인해야 하며, 포맷이 올바르고 유효한지 확인해야 함
- 스파크는 이런 복잡함을 피하고자 두 가지 종류의 시간 관련 정보만 집중적으로 관리함
  - 하나, 달력 형태의 날짜
  - 다른 하나, 날짜와 시간을 모두 가지는 타임스탬프
  - inferSchema 옵션이 활성화된 경우, 날짜와 타임스탬프를 포함해 컬럼의 데이터 타입을 최대한 정확하게 식별하려 시도함
    - 특정 날짜 포맷을 명시하지 않아도 자체적으로 식별해 데이터를 읽을 수 있음
- 날짜와 타임스탬프를 다루는 작업
  - 문자열을 다루는 작업과 관련이 있음
  - 날짜나 시간을 문자열로 저장하고 런타임에 날짜 타입으로 변환하는 경우가 많기 때문
  - 데이터베이스나 구조적 데이터를 다룰 때는 이러한 작업이 드물지만 텍스트나 CSV 파일을 다룰 때는 많이 발생함
  
  
- 스파크는 날짜와 시간을 최대한 올바른 형태로 읽기 위해 노력함
  - 만약 특이한 포맷의 날짜와 시간 데이터를 어쩔 수 없이 다뤄야 한다면 각 단계별로 어떤 데이터 타입과 포맷을 유지하는지 정확히 알고 트랜스포메이션을 적용해야 함
  - TimestampType 클래스 : 초 단위 정밀도까지만 지원함
  - Long 데이터 타입 :밀리세컨드나 마이크로 세컨드 단위를 다룰 때 해당 데이터 타입으로 변환해 처리하는 우회 정책을 사용해야 함


- 스파크는 특정 기점에 데이터 포맷이 약간 특이하게 변할 수 있음
  - 이러한 문제를 피하려면 파싱이나 변환 작업을 해야 함
  - 자바의 날짜와 타임스탬프를 사용해서 표준 체계를 따름

In [26]:
#오늘 날짜와 현재 타임스탬프값을 구하는 예제
from pyspark.sql.functions import current_date,current_timestamp
dateDF=spark.range(10)\
    .withColumn("today",current_date())\
    .withColumn("now",current_timestamp())
dateDF.createOrReplaceTempView("dateTable")
dateDF.printSchema()

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)



- 위 예제로 만들어진 DataFrame을 사용해 오늘을 기준으로 5일 전후의 날짜를 구해보겠음
  - date_sum 함수와 date_add 함수는 컬럼과 더하거나 뺄 날짜 수를 인수로 전달해야 함

In [27]:
from pyspark.sql.functions import date_add,date_sub
dateDF.select(date_sub(col("today"),5),date_add(col("today"),5)).show(1)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2021-11-14|        2021-11-24|
+------------------+------------------+
only showing top 1 row



- 두 날짜의 차이를 구하는 작업도 자주 발생함
  - datediff 함수 : 두 날짜 사이의 일 수를 반환
  - months_between 함수 : 두 날짜 사이의 개월 수를 반환

In [28]:
from pyspark.sql.functions import datediff,months_between,to_date

dateDF.withColumn("week_age",date_sub(col("today"),7))\
    .select(datediff(col("week_age"),col("today"))).show(1)
dateDF.select(
    to_date(lit("2016-01-01")).alias("start"),
    to_date(lit("2017-05-22")).alias("end"))\
    .select(months_between(col("start"),col("end"))).show(1)

+-------------------------+
|datediff(week_age, today)|
+-------------------------+
|                       -7|
+-------------------------+
only showing top 1 row

+--------------------------------+
|months_between(start, end, true)|
+--------------------------------+
|                    -16.67741935|
+--------------------------------+
only showing top 1 row



- to_date 함수
  - 문자열을 날짜로 변환할 수 있으며 필요에 따라 날짜 포맷도 함께 저장할 수 있음
  - SimpleDataFormat 클래스 : 함수의 날짜 포맷은 반드시 자바가 지원하는 포맷을 사용해야 함


In [29]:
from pyspark.sql.functions import to_date,lit
spark.range(5).withColumn("date",lit("2017-01-01"))\
    .select(to_date(col("date"))).show(1)

+-------------+
|to_date(date)|
+-------------+
|   2017-01-01|
+-------------+
only showing top 1 row



- 스파크는 날짜를 파싱할 수 없다면 에러 대신 null 값을 반환함
  - 다단계 처리 파이프라인에서는 조금 까다로울 수 있음
  - 데이터 포맷이 지정된 데이터에서 또 다른 포맷의 데이터가 나타날 수 있음
  

In [30]:
#이해를 돕기 위해 년-월-일 형태가 아닌 년-일-월 형태의 날짜 포맷을 사용하면
#스파크는 날짜를 파싱할 수 없어 null 값을 반환함
dateDF.select(to_date(lit("2016-20-12")),to_date(lit("2017-12-11"))).show(1)

+-------------------+-------------------+
|to_date(2016-20-12)|to_date(2017-12-11)|
+-------------------+-------------------+
|               null|         2017-12-11|
+-------------------+-------------------+
only showing top 1 row



- 지정한 날짜 형식에 맞춰 데이터가 들어온다면 특별한 문제가 발생하지 않음
  - 날짜 형식을 지키지 않ㅇ느 데이터가 들어온다면 디버깅하기 매우 어려움
  - ex)날짜(2017-12-11)가 의도한 날짜인 11월 12일 대신 12월 11일로 표시되고 있음
    - 스파크는 날짜가 뒤섞여 있거나 데이터가 잘못되었는지 판단할 수 없으므로 오류를 발생시키지 않음
- 위의 문제를 완전히 회피할 수 있는 견고한 방법
  - 첫 번째 단계 : 자바의 SimpleDateFormat 표준에 맞춰 날짜 포맷을 지정
    - to_date 함수 : 필요에 따라 날짜 포맷을 지정
    - to_timestamp 함수 : 반드시 날짜 포맷을 지정

In [31]:
from pyspark.sql.functions import to_date

dateFormat='yyyy-dd-MM'
cleanDateDF=spark.range(1).select(
    to_date(lit("2017-12-11"),dateFormat).alias("date"),
    to_date(lit("2017-20-12"),dateFormat).alias("date2"))
cleanDateDF.createOrReplaceTempView("dateTable2")

In [32]:
#to_timestamp항상 날짜 포맷을 지정해줘야 함
from pyspark.sql.functions import to_timestamp
cleanDateDF.select(to_timestamp(col("date"),dateFormat)).show()

+------------------------------+
|to_timestamp(date, yyyy-dd-MM)|
+------------------------------+
|           2017-11-12 00:00:00|
+------------------------------+



- 올바른 포맷과 타입의 날짜나 타임스탬프를 사용한다면 매우 쉽게 비교할 수 있음
  - 날짜를 비교할 때는 날짜나 타임스탬프 타입을 사용하거나 yyyy-MM-dd 포맷에 맞는 문자열을 지정함

In [33]:
cleanDateDF.filter(col("date2")>lit("2017-12-12")).show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



- 스파크가 리터럴로 인식하는 문자열을 지정해 날짜를 비교할 수도 있음

In [34]:
cleanDateDF.filter(col("date2")>"2017-12-12").show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



## 6.7 null 값 다루기
- DataFrame에서 빠져 있거나 비어 있는 데이터를 표현할 때는 항상 null 값을 사용하는 것이 좋음
  - 스파크에서는 빈 문자열이나 대체 값 대신 null 값을 사용해야 최적화를 수행할 수 있음
- DataFrame의 하위 패키지인 .na를 사용하는 것이 DataFrame에서 null 값을 다루는 기본 방식
  - 연산을 수행하면서 스파크가 null 값을 제어하는 방법을 명시적으로 지정하는 몇 가지 함수도 있음
  
- null 값을 다루는 두 가지 방법
  - 명시적으로 null 값 제거
  - 전역 또는 컬럼 단위로 null 값을 특정 값으로 채워 넣기
  
### 6.7.1 coalesce
- coalesce 함수 : 인수로 지정한 여러 컬럼 중 null이 아닌 첫 번째 값을 반환
  - 모든 컬럼이 null이 아닌 값을 가지는 경우 **첫 번째 컬럼의 값을 반환함**

In [35]:
from pyspark.sql.functions import coalesce
df.select(coalesce(col("Description"),col("CustomerId"))).show()

+---------------------------------+
|coalesce(Description, CustomerId)|
+---------------------------------+
|             WHITE HANGING HEA...|
|              WHITE METAL LANTERN|
|             CREAM CUPID HEART...|
|             KNITTED UNION FLA...|
|             RED WOOLLY HOTTIE...|
|             SET 7 BABUSHKA NE...|
|             GLASS STAR FROSTE...|
|             HAND WARMER UNION...|
|             HAND WARMER RED P...|
|             ASSORTED COLOUR B...|
|             POPPY'S PLAYHOUSE...|
|             POPPY'S PLAYHOUSE...|
|             FELTCRAFT PRINCES...|
|             IVORY KNITTED MUG...|
|             BOX OF 6 ASSORTED...|
|             BOX OF VINTAGE JI...|
|             BOX OF VINTAGE AL...|
|             HOME BUILDING BLO...|
|             LOVE BUILDING BLO...|
|             RECIPE BOX WITH M...|
+---------------------------------+
only showing top 20 rows



### 6.7.2 ifnull. nullif,nvl,nvl2
- coalesce 함수와 유사한 결과를 얻을 수 있는 몇 가지 SQL 함수
  - ifnull 함수
    - 첫 번째 값이 null이면 두 번째 값을 반환함
    - 첫 번째 값이 null이 아니면 첫 번째 값을 반환함
  - nullif 함수
    - 두 값이 같으면 null을 반환함
    - 두 값이 다르면 첫 번째 값을 반환함
  - nvl 함수
    - 첫 번째 값이 null이면 두 번째 값을 반환함
    - 첫 번째 값이 null이 아니면 첫 번째 값을 반환함
  - nvl2 함수
    - 첫 번째 값이 null이면 세 번째 인수로 지정된 값을 반환함
    - 첫 번째 값이 null이 아니면 두 번째 값을 반환함
- 위의 함수들은 DataFrame의 select 표현식으로 사용할 수 있음

### 6.7.3 drop
- drop 메서드
  - null 값을 가진 로우를 제거하는 가장 간단한 함수
  - 기본적으로 null 값을 가진 모든 로우 제거

In [36]:
df.na.drop()
df.na.drop("any")

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

- drop 메서드의 인수
  - any
    - 지정 시 로우의 컬럼값 중 하나라도 null 값을 가지면 해당 로우 제거
  - all
    - 모든 컬럼의 값이 null이거나 NaN인 경우에만 해당 로우 제거

In [37]:
df.na.drop("all")

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

- drop 메서드에 배열 형태의 컬럼을 인수로 전달해 적용할 수도 있음

In [38]:
df.na.drop("all",subset=['StockCode','InvoiceNo'])

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

### 6.7.4 fill
- fill 함수
  - 하나 이상의 컬럼을 특정 값으로 채울 수 있음
  - 채워 넣을 값과 컬럼 집합으로 구성된 맵을 인수로 사용함

In [39]:
#string 데이터 타입의 컬럼에 존재하는 null 값을 다른 값으로 채워 넣는 방법
df.na.fill("All Null values become this string")

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

- df.na.fill(5:Integer) 같은 방식을 사용해 Integer 데이터 타입의 컬럼에 존재하는 null 값을 다른 값으로 채워 넣을 수 있음
  - Double 데이터 타입의 컬럼에는 df.na.fill(5:double) 같은 방식을 사용할 수 있음
  - 다수의 컬럼에 적용하고 싶다면 적용하고자 하는 컬럼명을 배열로 만들어 인수로 사용함


In [40]:
df.na.fill("all",subset=['StockCode','InvoiceNo'])

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

- 파이썬의 경우, 딕셔너리 형태의 값을 하나 만들어서 넣으면 다수의 컬럼에 fill 메서드를 적용할 수 있음
  - 키 : 컬럼명, 값: null 값을 채우는 데 사용할 값


In [41]:
fill_cols_vals={"StockCode":5,"Description":"No Value"}
df.na.fill(fill_cols_vals)

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

### 6.7.5 replace
- drop 메서드와 fill 메서드 외에도 null 값을 유연하게 대처할 방법이 있음
  - 조건에 따라 다른 값으로 대체하는 것
- replace 메서드
  - 해당 메서드 사용 시 변경하고자 하는 값과 원래 값의 데이터 타입이 같아야 함

In [42]:
df.na.replace([""],["UNKNOWN"],"Description")

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

## 6.8 정렬하기
- 5장에서 설명한 것처럼???
  - asc_nulls_first, desc_nulls_first, asc_nulls_last, desc_nulls_last 함수를 사용해 DataFrame을 정렬할 때 null 값이 표시되는 기준을 지정할 수 있음

## 6.9 복합 데이터 타입 다루기
- 복합 데이터 타입을 사용하면 해결하려는 문제에 더욱 적합한 방식으로 데이터를 구성하고 구조화할 수 있음
  - 복합 데이터 타입 종류: 구조체, 배열, 맵
  
### 6.9.1 구조체
- 구조체 : DataFrame 내부의 DataFrame으로 생각할 수 있음
  - 쿼리문에서 다수의 컬럼을 괄호로 묶어 구조체를 만들 수 있음

In [45]:
#복합 데이터 타입을 가진 DataFrame 만들기
from pyspark.sql.functions import struct
complexDF=df.select(struct("Description","InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")

- 다른 DataFrame을 조회하는 것과 동일하게 사용할 수 있음
- 유일한 차이점은 문법에 점(.)을 사용하거나 getField 메서드를 사용한다는 것

In [48]:
complexDF.select("complex.Description")
complexDF.select(col("complex").getField("Description"))

DataFrame[complex.Description: string]

- \*문자를 사용해 모든 값을 조회할 수 있으며 모든 컬럼을 DataFrame의 최상위 수준으로 끌어올릴 수 있음

In [49]:
complexDF.select("complex.*")

DataFrame[Description: string, InvoiceNo: string]

### 6.9.2 배열
- 한 가지 사례 : 데이터에서 Description 컬럼의 모든 단어를 하나의 로우로 변환
  - Description 컬럼을 복합 데이터 타입인 배열로 변환

#### split
- split : 배열로 변환하는데 사용하는 함수
  - split 함수에 구분자를 인수로 전달해 배열로 변환함

In [50]:
from pyspark.sql.functions import split
df.select(split(col("Description")," ")).show(2)

+-------------------------+
|split(Description,  , -1)|
+-------------------------+
|     [WHITE, HANGING, ...|
|     [WHITE, METAL, LA...|
+-------------------------+
only showing top 2 rows



- split 함수 : 스파크에서 복합 데이터 타입을 마치 또 다른 컬럼처럼 다룰 수 있는 매우 강력한 기능
  - 파이썬과 유사한 문법을 사용해 배열값을 조회할 수 있음

In [51]:
df.select(split(col("Description")," ").alias("array_col"))\
    .selectExpr("array_col[0]").show(2)

+------------+
|array_col[0]|
+------------+
|       WHITE|
|       WHITE|
+------------+
only showing top 2 rows



#### 배열의 길이
- 배열의 크기를 조회해 배열의 길이를 알 수 있음

In [52]:
from pyspark.sql.functions import size
df.select(size(split(col("Description")," "))).show(2)

+-------------------------------+
|size(split(Description,  , -1))|
+-------------------------------+
|                              5|
|                              3|
+-------------------------------+
only showing top 2 rows



#### array_contains
- array_contains 함수 : 배열에 특정 값이 존재하는지 확인


In [54]:
from pyspark.sql.functions import array_contains
df.select(array_contains(split(col("Description")," "),"WHITE")).show(2)

+------------------------------------------------+
|array_contains(split(Description,  , -1), WHITE)|
+------------------------------------------------+
|                                            true|
|                                            true|
+------------------------------------------------+
only showing top 2 rows



#### explode
- 복합 데이터 타입의 배열에 존재하는 모든 값을 로우로 변환
- 배열 타입의 컬럼을 입력받음
  - 입력된 컬럼의 배열값에 포함된 모든 값을 로우로 변환함
  - 나머지 컬럼값은 중복되어 표시됨

In [55]:
from pyspark.sql.functions import split,explode
df.withColumn("splitted",split(col("Description")," "))\
    .withColumn("exploded",explode(col("splitted")))\
    .select("Description","InvoiceNo","exploded").show(2)

+--------------------+---------+--------+
|         Description|InvoiceNo|exploded|
+--------------------+---------+--------+
|WHITE HANGING HEA...|   536365|   WHITE|
|WHITE HANGING HEA...|   536365| HANGING|
+--------------------+---------+--------+
only showing top 2 rows



### 6.9.3 맵
- 맵 : map 함수와 컬럼의 키-값 쌍을 이용해 생성함
  - 배열과 동일한 방법으로 값을 선택할 수 있음

In [57]:
from pyspark.sql.functions import create_map
df.select(create_map(col("Description"),col("InvoiceNo")).alias("complex_map")).show(2)

+--------------------+
|         complex_map|
+--------------------+
|{WHITE HANGING HE...|
|{WHITE METAL LANT...|
+--------------------+
only showing top 2 rows



- 적합한 키를 사용해 데이터를 조회할 수 있으며 해당 키가 없다면 null 값을 반환함

In [58]:
df.select(create_map(col("Description"),col("InvoiceNo")).alias("complex_map"))\
    .selectExpr("complex_map['WHITE METAL LANTERN']").show(2)

+--------------------------------+
|complex_map[WHITE METAL LANTERN]|
+--------------------------------+
|                            null|
|                          536365|
+--------------------------------+
only showing top 2 rows



- map 타입은 분해하여 컬럼으로 변환할 수 있음

In [59]:
df.select(create_map(col("Description"),col("InvoiceNo")).alias("complex_map"))\
    .selectExpr("explode(complex_map)").show(2)

+--------------------+------+
|                 key| value|
+--------------------+------+
|WHITE HANGING HEA...|536365|
| WHITE METAL LANTERN|536365|
+--------------------+------+
only showing top 2 rows



## 6.10 JSON 다루기
- 스파크는 JSON 데이터를 다루기 위한 몇 가지 고유 기능을 지원함
  - 문자열 형태의 JSON을 직접 조작할 수 있으며 JSON을 파싱하거나 JSON 객체로 만들 수 있음

In [60]:
#JSON 컬럼을 생성하는 예제
jsonDF=spark.range(1).selectExpr("""
'{"myJSONKey":{"myJSONValue":[1,2,3]}}' as JsonString""")